In [6]:
import numpy as np
import matplotlib.pyplot as plt
from ising_tri_2 import IsingSim_2
from joblib import Parallel, delayed
np.random.seed(0)

target_hist = np.load('histogram_surface_atoms.npy')


target_hist_mod = rot_inv_hist(hist = target_hist, array  = rot_array)

def stat_dist(hist_1, hist_2):
    s = (np.dot(np.sqrt(hist_1),np.sqrt(hist_2)))
    a = (1-s)  # Cause we are finding the global maximum using GP
    return a

In [7]:
from GPim.Gpim import gprutils as utils
from GPim.Gpim.gpreg.gpr import reconstructor
from GPim.Gpim.gpreg.skgpr import skreconstructor
from GPim.Gpim.gpreg.vgpr import vreconstructor
from GPim.Gpim.gpbayes.boptim import boptimizer

In [8]:
def J2_to_S_func2(indices):
    st = np.zeros(len(indices))
    J_mat = np.zeros([len(st),3,5])
    for i,idx in enumerate(indices):
      Jci, Jsi, Jdi = idx[0],idx[1],idx[2]

      Jc = 0.1*Jci - 3
      Js = 0.1*Jsi - 3
      Jd = 0.1*Jdi - 3

      J_mat[i] = np.array([[0,Js,0,Jd,0],
                          [Jc,0,0,0,Jc],
                          [0,Jd,0,Js,0]])
    
    histogram = Parallel(n_jobs=-1)(delayed(perform)(J) for J in J_mat)
    for i,hist in enumerate(histogram):
#         hist_mod = rot_inv_hist(hist = hist, array  = rot_array)
        st[i] = stat_dist(target_hist, hist).astype(float)
    return st

def perform(Jmat):
    ising_model =  IsingSim_2(N = 30, J_mat = Jmat, save_trajectories=True, T = 0.8, eqSteps =  750, mcSteps = 750, prop = 0.19)
    ising_model.performIsingSim()
    results = ising_model.results
    histogram = results['Histogram']
    
    return histogram

In [9]:
size_Jc, size_Js, size_Jd = 60, 60, 60
Z_sparse = np.ones((size_Jc, size_Js, size_Jd))*np.nan

idx = np.random.randint(0, Z_sparse.shape[0], size=(5, 5))

seeds = []
for i in range(5):
  seeds.append(np.array((idx[0,i], idx[1,i], idx[2,i])).T) 

A = J2_to_S_func2(seeds)

print(A)
for i, seed in enumerate(seeds):
    Z_sparse[tuple(seed)] = A[i]

# plt.figure(figsize=(6, 6))
# plt.imshow(Z_sparse[:,:])
# plt.suptitle('Seed points')

[0.37030109 0.36866415 0.41166351 0.23468656 0.2456374 ]


In [10]:
import random

def acq(gpmodel, X_full, X_sparse):  # leave it as is
    mean, sd = gpmodel.predict(X_full, verbose=0) # leave it as is
    acq = 0*mean + 1 * sd
    return acq, (mean, sd)  # leave it as is

def acq0(gpmodel, X_full, X_sparse):  # leave it as is
    mean, sd = gpmodel.predict(X_full, verbose=0) # leave it as is
    random_bit = random.getrandbits(1)
    random_boolean = bool(random_bit)
    if random_boolean:
      acq = -1.0 * mean + 1.0 * sd
    else:
      acq = 0 * mean + 1.0 * sd
    return acq, (mean, sd)  # leave it as is

def acq1(gpmodel, X_full, X_sparse):  # leave it as is
    mean, sd = gpmodel.predict(X_full, verbose=0) # leave it as is
    acq = np.exp(-(mean-1)**2)
    return acq, (mean, sd)  # leave it as is

def acq2(gpmodel, X_full, X_sparse):  # leave it as is
    mean, sd = gpmodel.predict(X_full, verbose=0) # leave it as is
    random_bit = np.random.rand()
    
    if random_bit < 0.60:
      acq = np.exp(-5*(mean-0)**2)
    else:
      acq = sd
    return acq, (mean, sd)  # leave it as is

In [ ]:
# Get full and sparse grids
X_full = utils.get_full_grid(Z_sparse)
X_sparse= utils.get_sparse_grid(Z_sparse)
# Initialize Bayesian optimizer with a custom acquisition function
boptim = boptimizer(
    X_sparse, Z_sparse, X_full, 
    J2_to_S_func2, acquisition_function=acq2,  # added custom acquisition function
    exploration_steps = 100, batch_update = True, batch_size = 20,
    dscale = 1,  # added distance-based criteria for point selection
    use_gpu = False, verbose=1)
# Run Bayesian optimization
boptim.run()


Exploration step 1 / 100
Model training...
average time per iteration: 0.0393 s
training completed in 37.35 s
Final parameter values:
 amp: 0.0559, lengthscale: [29.997  29.9841 29.9983], noise: 2.28e-05
Computing acquisition function...
indices
[[0, 59, 59], [0, 57, 59], [0, 58, 58], [1, 58, 59], [0, 59, 57], [1, 59, 58], [2, 59, 59], [0, 55, 59], [0, 56, 58], [1, 56, 59]]


In [12]:
vals_all = np.array(boptim.vals_all)
func_val, gp_pred = boptim.target_func_vals, boptim.gp_predictions
inds_all = np.array(boptim.indices_all)

In [13]:
results_3d = {}
results_3d['gp_pred'] = gp_pred
results_3d['func_val'] = func_val
results_3d['inds_all'] = inds_all
results_3d['vals_all'] = vals_all

In [ ]:
import pickle
pickle.dump( results_3d, open( "results_3d.p", "wb" ) )